# FFIEC Data Connect - Comprehensive Demo

This notebook demonstrates all the capabilities of the FFIEC Data Connect library, including:
- Basic data collection (sync)
- Async data collection with rate limiting
- Parallel processing for multiple data requests
- Working with pandas and polars DataFrames
- Error handling and validation
- Performance comparisons

**Note**: You'll need valid FFIEC credentials to run the live data examples. Mock examples are provided for demonstration.

# FFIEC Data Connect - Comprehensive Demo

This notebook demonstrates all the capabilities of the FFIEC Data Connect library, including:
- Basic data collection (sync)
- Async data collection with rate limiting
- Parallel processing for multiple data requests
- Working with pandas and polars DataFrames
- Error handling and validation
- Performance comparisons

## 🚀 Quick Start

1. **First**: Run the installation cell above to install all required packages
2. **Second**: Enter your FFIEC credentials when prompted (or set environment variables)
3. **Then**: Run all remaining cells to see the full demonstration

## 📋 Requirements

- Python 3.8+
- Valid FFIEC CDR credentials (or demo mode will be used)
- Internet connection for package installation

**Note**: If you have FFIEC credentials, the notebook will collect real data. Otherwise, it will demonstrate all features using mock data.

## 📦 Setup and Imports

In [ ]:
# Standard library imports
import os
import asyncio
import time
import random
from datetime import datetime, timedelta
from typing import List, Dict, Any

# Third-party imports
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# FFIEC Data Connect imports
import ffiec_data_connect as fdc
from ffiec_data_connect import (
    WebserviceCredentials,
    FFIECConnection,
    AsyncCompatibleClient,
    RateLimiter,
    collect_data,
    collect_reporting_periods,
    collect_filers_on_reporting_period,
    CredentialError,
    ValidationError,
    NoDataError
)

print(f"FFIEC Data Connect version: {fdc.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Polars version: {pl.__version__}")

In [ ]:
# Create an AsyncCompatibleClient instance
client = AsyncCompatibleClient(
    max_concurrent=3,      # Maximum concurrent requests
    rate_limit=2.0         # Rate limit: 2 seconds between requests
)

In [ ]:
# Method 1: Environment variables (recommended for production)
# Set these in your environment:
# export FFIEC_USERNAME="your_username"
# export FFIEC_PASSWORD="your_password"

try:
    # This will use environment variables if available
    credentials = WebserviceCredentials()
    print("✅ Credentials loaded from environment variables")
    print(f"Username: {credentials.masked_username}")
    
except CredentialError:
    print("❌ No credentials found in environment variables")
    print("For live data examples, you'll need to set FFIEC_USERNAME and FFIEC_PASSWORD")
    print("Using mock mode for demonstrations...")
    
    # Create mock credentials for demo purposes
    credentials = WebserviceCredentials("demo_user", "demo_pass")
    DEMO_MODE = True
else:
    DEMO_MODE = False

# Method 2: Direct instantiation (for testing/development)
# credentials = WebserviceCredentials("your_username", "your_password")

# Method 3: Interactive (uncomment to use)
# import getpass
# username = input("FFIEC Username: ")
# password = getpass.getpass("FFIEC Password: ")
# credentials = WebserviceCredentials(username, password)

## 🔌 Connection Management

The library provides multiple connection management approaches:
- Basic FFIECConnection
- AsyncCompatibleClient with advanced features
- Context managers for automatic cleanup

In [ ]:
# Basic connection
basic_connection = FFIECConnection()
print(f"Basic connection: {basic_connection}")
print(f"Session active: {basic_connection.session is not None}")

# Advanced async-compatible client with rate limiting
async_client = AsyncCompatibleClient(
    credentials=credentials,
    max_concurrent=4,  # Max 4 concurrent requests
    rate_limit=0.5  # Max 0.5 requests per second (30 per minute)
)

print(f"\nAsync client: {async_client}")
print(f"Rate limiter: {async_client.rate_limiter}")

# Context manager usage (recommended)
print("\n🔄 Testing context manager...")
with AsyncCompatibleClient(credentials) as client:
    print(f"Client active: {len(client._connection_cache) >= 0}")  # Check cache exists
print("Client automatically closed after context")

## 📊 Basic Data Collection (Synchronous)

Start with the traditional synchronous API for collecting FFIEC data.

In [ ]:
# Mock server setup for demonstration
if DEMO_MODE:
    from tests.mocks.soap_server import MockFFIECServer
    
    # Start mock server for demo
    mock_server = MockFFIECServer(host='localhost', port=8089)
    mock_server.start()
    print("🚀 Mock FFIEC server started for demonstration")
    print(f"Mock server URL: {mock_server.url}")
    
    # Patch the connection to use mock server
    import ffiec_data_connect.constants as constants
    constants.WebserviceConstants.base_url = mock_server.wsdl_url
    
    # Sample data for demos
    SAMPLE_BANKS = ["480228", "852320", "628403"]  # JPMorgan Chase, Bank of America, Wells Fargo
    SAMPLE_PERIODS = ["2023-12-31", "2023-09-30", "2023-06-30"]
else:
    # Real data examples (you'll need valid credentials)
    SAMPLE_BANKS = ["480228", "852320"]  # Use fewer banks for real API
    SAMPLE_PERIODS = ["2023-12-31", "2023-09-30"]

print(f"Sample banks (RSSD IDs): {SAMPLE_BANKS}")
print(f"Sample periods: {SAMPLE_PERIODS}")

In [ ]:
# 1. Get available reporting periods
print("📅 Getting available reporting periods...")

try:
    periods = collect_reporting_periods(
        session=basic_connection.session,
        creds=credentials,
        series="call",
        output_type="list"
    )
    
    print(f"Found {len(periods)} reporting periods")
    print("Recent periods:")
    for period in periods[:5]:  # Show first 5
        print(f"  - {period}")
        
except Exception as e:
    print(f"Error getting periods: {e}")
    periods = SAMPLE_PERIODS  # Fallback

In [ ]:
# 2. Collect data for a single bank
print("🏦 Collecting data for a single bank...")

rssd_id = SAMPLE_BANKS[0]  # JPMorgan Chase
reporting_period = SAMPLE_PERIODS[0]  # Most recent

print(f"Bank RSSD ID: {rssd_id}")
print(f"Reporting period: {reporting_period}")

try:
    start_time = time.time()
    
    bank_data = collect_data(
        session=basic_connection.session,
        creds=credentials,
        reporting_period=reporting_period,
        rssd_id=rssd_id,
        series="call",
        output_type="list"
    )
    
    elapsed = time.time() - start_time
    
    print(f"✅ Collected {len(bank_data)} data points in {elapsed:.2f} seconds")
    
    # Show sample data
    if bank_data:
        print("\nSample data points:")
        for i, item in enumerate(bank_data[:3]):
            print(f"  {i+1}. {item}")
            
except Exception as e:
    print(f"❌ Error collecting data: {e}")
    bank_data = []

## ⚡ Async Data Collection

The AsyncCompatibleClient provides async methods for better performance when collecting multiple datasets.

In [ ]:
# Async data collection with rate limiting
async def collect_data_async_demo():
    """Demo async data collection with rate limiting."""
    
    # Use async context manager
    async with AsyncCompatibleClient(
        credentials=credentials,
        max_concurrent=5,
        rate_limit=2.0  # 2 requests per second
    ) as client:
        
        print("🚀 Starting async data collection...")
        start_time = time.time()
        
        # Collect data for multiple banks async
        tasks = []
        for rssd_id in SAMPLE_BANKS[:2]:  # First 2 banks
            task = client.collect_data_async(
                reporting_period=SAMPLE_PERIODS[0],
                rssd_id=rssd_id,
                series="call"
            )
            tasks.append((rssd_id, task))
        
        # Wait for all tasks to complete
        results = []
        for rssd_id, task in tasks:
            try:
                data = await task
                results.append((rssd_id, data))
                print(f"✅ Bank {rssd_id}: {len(data)} data points")
            except Exception as e:
                print(f"❌ Bank {rssd_id}: Error - {e}")
        
        elapsed = time.time() - start_time
        print(f"\n⏱️ Async collection completed in {elapsed:.2f} seconds")
        return results

# Run the async demo
try:
    async_results = await collect_data_async_demo()
except Exception as e:
    print(f"❌ Async demo error: {e}")
    async_results = []

## 🔄 Parallel Processing (Sync Interface)

For users who prefer synchronous code, the library provides parallel processing with a sync interface.

In [ ]:
# Parallel data collection with sync interface
print("🔄 Parallel data collection demo...")

with AsyncCompatibleClient(credentials, max_concurrent=3) as client:
    
    print(f"Collecting data for multiple banks in parallel...")
    
    # Progress callback function
    def progress_callback(rssd_id: str, result: Any):
        if 'error' in str(result):
            print(f"❌ Bank {rssd_id}: Error")
        else:
            data_points = len(result) if isinstance(result, list) else 0
            print(f"✅ Bank {rssd_id}: {data_points} data points")
    
    try:
        start_time = time.time()
        
        # Use parallel collection method with correct signature
        results = client.collect_data_parallel(
            reporting_period=SAMPLE_PERIODS[0],  # Single period
            rssd_ids=SAMPLE_BANKS[:2],  # Multiple banks
            series='call',
            progress_callback=progress_callback
        )
        
        elapsed = time.time() - start_time
        
        # Process results
        successful = [r for r in results.values() if 'error' not in str(r)]
        failed = [r for r in results.values() if 'error' in str(r)]
        
        print(f"\n✅ Parallel collection completed in {elapsed:.2f} seconds")
        print(f"📈 Successful: {len(successful)}, Failed: {len(failed)}")
        
        if successful:
            total_data_points = sum(len(r) if isinstance(r, list) else 0 for r in successful)
            print(f"📊 Total data points collected: {total_data_points}")
            
    except Exception as e:
        print(f"❌ Parallel collection error: {e}")
        results = {}

## Data Type Consistency with Polars

The library ensures proper numpy dtypes throughout the data pipeline from XBRL processing to pandas DataFrames to polars conversion. Let's convert the pandas DataFrame to polars and examine the type consistency.

In [ ]:
# Direct XBRL to polars conversion - preserves maximum precision
try:
    df_polars_direct = collect_data(
        session=basic_connection.session,
        creds=credentials, 
        reporting_period="2023-12-31",
        rssd_id="480228",
        series="call",
        output_type="polars"  # Direct conversion from XBRL to polars
    )
    
    print("Direct XBRL → polars DataFrame:")
    print(f"Shape: {df_polars_direct.shape}")
    print(f"Schema: {df_polars_direct.schema}")
    print("\nFirst few rows:")
    print(df_polars_direct.head())
    
except Exception as e:
    print(f"❌ Error creating polars DataFrame: {e}")
    # Create empty DataFrame for demo
    import polars as pl
    df_polars_direct = pl.DataFrame({
        'mdrm': ['DEMO'],
        'rssd': ['480228'], 
        'quarter': ['2023-12-31'],
        'data_type': ['int'],
        'int_data': [1000000],
        'float_data': [None],
        'bool_data': [None],
        'str_data': [None]
    })

In [ ]:
# Create pandas DataFrame for comparison
try:
    df_pandas = collect_data(
        session=basic_connection.session,
        creds=credentials,
        reporting_period="2023-12-31", 
        rssd_id="480228",
        series="call",
        output_type="pandas"  # Standard pandas conversion
    )
    
    print("=== Type Precision Comparison ===")
    print("\nPandas DataFrame (via pandas conversion):")
    print(df_pandas.dtypes)
    
    if len(df_pandas) > 0 and 'int_data' in df_pandas.columns:
        int_samples = df_pandas.dropna(subset=['int_data'])
        if len(int_samples) > 0:
            sample_pandas = int_samples['int_data'].iloc[0]
            print(f"Sample int value: {sample_pandas} (type: {type(sample_pandas)})")
        
        print("\nPolars DataFrame (direct from XBRL):")  
        print(df_polars_direct.schema)
        
        if len(df_polars_direct) > 0:
            polars_ints = df_polars_direct.filter(pl.col('int_data').is_not_null())
            if len(polars_ints) > 0:
                sample_polars = polars_ints['int_data'].first()
                print(f"Sample int value: {sample_polars} (type: {type(sample_polars)})")
                
                # Verify no precision loss in the direct conversion
                int_data_polars = polars_ints['int_data'].to_list()
                int_data_pandas = int_samples['int_data'].tolist()
                
                print(f"\nPrecision check - values match: {int_data_polars == int_data_pandas}")
            
except Exception as e:
    print(f"❌ Error in comparison: {e}")
    # Create demo pandas DataFrame
    import pandas as pd
    df_pandas = pd.DataFrame({
        'mdrm': ['DEMO'],
        'rssd': ['480228'],
        'quarter': ['2023-12-31'], 
        'data_type': ['int'],
        'int_data': [1000000],
        'float_data': [None],
        'bool_data': [None],
        'str_data': [None]
    })
    print("Using demo data for comparison")

In [ ]:
# Save pandas data to various formats
print("💾 Saving pandas data to files...")

# Save to CSV
csv_file = 'ffiec_data_pandas.csv'
df_pandas.to_csv(csv_file, index=False)
print(f"✅ Saved to CSV: {csv_file}")

# Save to Excel
excel_file = 'ffiec_data_pandas.xlsx'
with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
    df_pandas.to_excel(writer, sheet_name='All_Data', index=False)
    
    # Create separate sheets by metric
    if 'metric' in df_pandas.columns:
        for metric in df_pandas['metric'].unique():
            metric_data = df_pandas[df_pandas['metric'] == metric]
            safe_sheet_name = metric.replace(' ', '_')[:31]  # Excel sheet name limits
            metric_data.to_excel(writer, sheet_name=safe_sheet_name, index=False)

print(f"✅ Saved to Excel: {excel_file}")

# Save to Parquet (efficient for large datasets)
parquet_file = 'ffiec_data_pandas.parquet'
df_pandas.to_parquet(parquet_file, index=False)
print(f"✅ Saved to Parquet: {parquet_file}")

# Display file sizes
import os
print("\n📏 File sizes:")
for file in [csv_file, excel_file, parquet_file]:
    if os.path.exists(file):
        size_mb = os.path.getsize(file) / (1024 * 1024)
        print(f"  {file}: {size_mb:.2f} MB")

In [ ]:
# Display the schemas to verify type consistency
print("=== Final Schema Comparison ===")

if 'df_pandas' in locals():
    print("\nPandas DataFrame dtypes:")
    print(df_pandas.dtypes)
else:
    print("\n❌ Pandas DataFrame not available")

if 'df_polars_direct' in locals():
    print("\nPolars DataFrame schema (direct conversion):")
    print(df_polars_direct.schema)
    
    # Show some sample data with proper types
    if len(df_polars_direct) > 0:
        int_vals = df_polars_direct.filter(pl.col('int_data').is_not_null())
        float_vals = df_polars_direct.filter(pl.col('float_data').is_not_null())
        bool_vals = df_polars_direct.filter(pl.col('bool_data').is_not_null())
        
        if len(int_vals) > 0:
            sample_int = int_vals['int_data'].first()
            print(f"\nSample integer value: {sample_int} (type: {type(sample_int)})")
            
        if len(float_vals) > 0:
            sample_float = float_vals['float_data'].first()
            print(f"Sample float value: {sample_float} (type: {type(sample_float)})")
            
        if len(bool_vals) > 0:
            sample_bool = bool_vals['bool_data'].first()
            print(f"Sample boolean value: {sample_bool} (type: {type(sample_bool)})")
        else:
            print("\nNo boolean data in this dataset")
            
    print(f"\n✅ Direct polars conversion preserves {len(df_polars_direct.schema)} column types")
else:
    print("\n❌ Polars DataFrame not available")

In [ ]:
# Create Polars DataFrame from the same data
print("⚡ Creating Polars DataFrame...")

# Convert from pandas or create directly
df_polars = pl.from_pandas(df_pandas)

# Or create directly from data
# df_polars = pl.DataFrame(pandas_data)

print(f"Created Polars DataFrame with {len(df_polars)} rows")
print(f"Columns: {df_polars.columns}")
print(f"Schema: {df_polars.schema}")

# Display sample data
print("\n📊 Sample data (Polars):")
print(df_polars.head(10))

In [ ]:
# Polars data analysis examples
print("⚡ Polars Analysis Examples:")

# Check the actual data structure to provide meaningful analysis
print(f"\nAvailable columns: {df_polars.columns}")
print(f"Data types in the data: {df_polars['data_type'].unique().to_list() if 'data_type' in df_polars.columns else 'No data_type column'}")

# Create meaningful analysis based on the actual data structure
if len(df_polars) > 0 and 'int_data' in df_polars.columns:
    
    # 1. Basic aggregation by data type
    print("\n📊 Data summary by type:")
    type_summary = (
        df_polars
        .group_by('data_type')
        .agg([
            pl.count().alias('count'),
            pl.col('mdrm').n_unique().alias('unique_mdrms')
        ])
        .sort('data_type')
    )
    print(type_summary)
    
    # 2. Integer data analysis (if available)
    int_data = df_polars.filter(pl.col('int_data').is_not_null())
    if len(int_data) > 0:
        print(f"\n💰 Integer data analysis ({len(int_data)} records):")
        int_stats = (
            int_data
            .group_by(['rssd', 'quarter'])
            .agg([
                pl.col('int_data').min().alias('min_value'),
                pl.col('int_data').max().alias('max_value'),
                pl.col('int_data').mean().alias('avg_value'),
                pl.col('int_data').count().alias('count')
            ])
            .sort('rssd')
        )
        print(int_stats)
        
        # Top 5 largest values
        top_values = (
            int_data
            .sort('int_data', descending=True)
            .select(['mdrm', 'rssd', 'quarter', 'int_data'])
            .head(5)
        )
        print(f"\n🏆 Top 5 largest integer values:")
        print(top_values)
    
    # 3. Float data analysis (if available)
    float_data = df_polars.filter(pl.col('float_data').is_not_null())
    if len(float_data) > 0:
        print(f"\n📈 Float data analysis ({len(float_data)} records):")
        float_stats = (
            float_data
            .group_by('rssd')
            .agg([
                pl.col('float_data').min().alias('min_ratio'),
                pl.col('float_data').max().alias('max_ratio'),
                pl.col('float_data').mean().alias('avg_ratio'),
                pl.col('float_data').count().alias('count')
            ])
            .sort('avg_ratio', descending=True)
        )
        print(float_stats)
    
    # 4. MDRM code frequency analysis
    print(f"\n📋 Most common MDRM codes:")
    mdrm_freq = (
        df_polars
        .group_by('mdrm')
        .agg([
            pl.count().alias('frequency'),
            pl.col('data_type').first().alias('data_type')
        ])
        .sort('frequency', descending=True)
        .head(10)
    )
    print(mdrm_freq)
    
    # 5. Data distribution by bank and quarter
    print(f"\n🏦 Data points by bank and quarter:")
    bank_quarter_summary = (
        df_polars
        .group_by(['rssd', 'quarter'])
        .agg([
            pl.count().alias('total_data_points'),
            pl.col('data_type').n_unique().alias('unique_data_types')
        ])
        .sort(['rssd', 'quarter'])
    )
    print(bank_quarter_summary)

else:
    print("\n📝 This dataset uses the direct XBRL structure with type-specific columns:")
    print("   - int_data: Integer financial values")
    print("   - float_data: Ratio and percentage values") 
    print("   - bool_data: Boolean indicators")
    print("   - str_data: Text values")
    print("\n💡 This design preserves maximum type precision from the XBRL source")
    print("   and avoids the precision loss that can occur with generic 'value' columns.")

In [ ]:
# Save Polars data to various formats
print("💾 Saving Polars data to files...")

# Polars CSV (very fast)
polars_csv = 'ffiec_data_polars.csv'
df_polars.write_csv(polars_csv)
print(f"✅ Saved to CSV: {polars_csv}")

# Polars Parquet (recommended for large datasets)
polars_parquet = 'ffiec_data_polars.parquet'
df_polars.write_parquet(polars_parquet)
print(f"✅ Saved to Parquet: {polars_parquet}")

# JSON (good for smaller datasets)
polars_json = 'ffiec_data_polars.json'
df_polars.write_json(polars_json)
print(f"✅ Saved to JSON: {polars_json}")

# Delta format (if delta-rs is installed)
try:
    df_polars.write_delta('ffiec_data_delta')
    print(f"✅ Saved to Delta format")
except Exception as e:
    print(f"ℹ️ Delta format not available: {e}")

# Display file sizes comparison
print("\n📏 File sizes comparison:")
polars_files = [polars_csv, polars_parquet, polars_json]
for file in polars_files:
    if os.path.exists(file):
        size_mb = os.path.getsize(file) / (1024 * 1024)
        print(f"  {file}: {size_mb:.2f} MB")

## ⚡ Performance Comparison

Compare the performance of different approaches for data collection and processing.

In [ ]:
# Performance comparison of sync vs async vs parallel
print("🏁 Performance Comparison")

# Create test data for meaningful comparison
PERFORMANCE_BANKS = SAMPLE_BANKS[:3]  # Use first 3 banks for testing
PERFORMANCE_PERIOD = SAMPLE_PERIODS[0]  # Use most recent period

print(f"Testing with {len(PERFORMANCE_BANKS)} banks for period {PERFORMANCE_PERIOD}")

# Mock the collect_data function for performance testing
def mock_collect_data(session, creds, reporting_period, rssd_id, series="call", **kwargs):
    """Mock function that simulates data collection delay."""
    import random
    import time
    
    # Simulate network delay
    time.sleep(random.uniform(0.1, 0.3))
    
    # Return mock data
    return [{
        'rssd_id': rssd_id,
        'reporting_period': reporting_period,
        'metric': 'Total Assets',
        'value': random.randint(10000, 500000),
        'mdrm': 'RCON2170'
    }]

performance_results = {}

In [ ]:
# Test 1: Sequential (traditional approach)
print("🐌 Testing sequential collection...")

start_time = time.time()
sequential_results = []

for i, rssd_id in enumerate(PERFORMANCE_BANKS):
    if DEMO_MODE:
        result = mock_collect_data(
            session=basic_connection.session,
            creds=credentials,
            reporting_period=PERFORMANCE_PERIOD,
            rssd_id=rssd_id,
            series="call"
        )
    else:
        try:
            result = collect_data(
                session=basic_connection.session,
                creds=credentials,
                reporting_period=PERFORMANCE_PERIOD,
                rssd_id=rssd_id,
                series="call"
            )
        except Exception:
            result = mock_collect_data(
                session=basic_connection.session,
                creds=credentials,
                reporting_period=PERFORMANCE_PERIOD,
                rssd_id=rssd_id,
                series="call"
            )
    
    sequential_results.append(result)
    print(f"  Completed {i+1}/{len(PERFORMANCE_BANKS)}")

sequential_time = time.time() - start_time
performance_results['Sequential'] = sequential_time

print(f"✅ Sequential: {sequential_time:.2f} seconds for {len(PERFORMANCE_BANKS)} requests")
print(f"   Average: {sequential_time/len(PERFORMANCE_BANKS):.2f} seconds per request")

In [ ]:
# Test 2: Parallel (sync interface)
print("🚀 Testing parallel collection...")

# Ensure sequential_time is available for comparison
if 'sequential_time' not in locals():
    print("⚠️ Sequential test not run yet - using estimated baseline")
    sequential_time = 1.0  # Fallback value

with AsyncCompatibleClient(credentials, max_concurrent=4) as client:
    start_time = time.time()
    
    # Mock the client's collect_data method for testing
    if DEMO_MODE:
        original_method = client.collect_data
        def mock_client_collect_data(reporting_period, rssd_id, series="call", **kwargs):
            return mock_collect_data(
                session=basic_connection.session,
                creds=credentials,
                reporting_period=reporting_period,
                rssd_id=rssd_id,
                series=series,
                **kwargs
            )
        client.collect_data = mock_client_collect_data
    
    try:
        parallel_results = client.collect_data_parallel(
            reporting_period=PERFORMANCE_PERIOD,
            rssd_ids=PERFORMANCE_BANKS,
            series="call"
        )
        
        parallel_time = time.time() - start_time
        performance_results['Parallel'] = parallel_time
        
        print(f"✅ Parallel: {parallel_time:.2f} seconds for {len(PERFORMANCE_BANKS)} requests")
        print(f"   Average: {parallel_time/len(PERFORMANCE_BANKS):.2f} seconds per request")
        
        # Only calculate speedup if we have sequential_time
        if 'sequential_time' in locals() and sequential_time > 0:
            speedup = sequential_time / parallel_time
            print(f"   Speedup: {speedup:.1f}x faster than sequential")
        else:
            print("   (Sequential time not available for comparison)")
        
    except Exception as e:
        print(f"❌ Parallel test failed: {e}")
        performance_results['Parallel'] = float('inf')
    
    finally:
        # Restore original method
        if DEMO_MODE and 'original_method' in locals():
            client.collect_data = original_method

In [ ]:
# Test 3: Pure async (for comparison)
print("⚡ Testing pure async collection...")

# Ensure sequential_time is available for comparison
if 'sequential_time' not in locals():
    print("⚠️ Sequential test not run yet - using estimated baseline")
    sequential_time = 1.0  # Fallback value

async def test_async_performance():
    async with AsyncCompatibleClient(
        credentials, 
        max_concurrent=5,
        rate_limit=20.0  # 20 requests per second
    ) as client:
        
        start_time = time.time()
        
        # Create async tasks
        tasks = []
        for rssd_id in PERFORMANCE_BANKS:
            if DEMO_MODE:
                # Create mock async task
                async def mock_async(rssd_id=rssd_id):
                    await asyncio.sleep(random.uniform(0.1, 0.3))
                    return mock_collect_data(
                        session=basic_connection.session,
                        creds=credentials,
                        reporting_period=PERFORMANCE_PERIOD,
                        rssd_id=rssd_id,
                        series="call"
                    )
                task = mock_async()
            else:
                task = client.collect_data_async(
                    reporting_period=PERFORMANCE_PERIOD,
                    rssd_id=rssd_id,
                    series="call"
                )
            tasks.append(task)
        
        # Execute all tasks concurrently
        async_results = await asyncio.gather(*tasks, return_exceptions=True)
        
        async_time = time.time() - start_time
        
        return async_time, async_results

try:
    async_time, async_results = await test_async_performance()
    performance_results['Async'] = async_time
    
    print(f"✅ Async: {async_time:.2f} seconds for {len(PERFORMANCE_BANKS)} requests")
    print(f"   Average: {async_time/len(PERFORMANCE_BANKS):.2f} seconds per request")
    
    # Only calculate speedup if we have sequential_time
    if 'sequential_time' in locals() and sequential_time > 0:
        speedup = sequential_time / async_time
        print(f"   Speedup: {speedup:.1f}x faster than sequential")
    else:
        print("   (Sequential time not available for comparison)")
    
except Exception as e:
    print(f"❌ Async test failed: {e}")
    performance_results['Async'] = float('inf')

In [ ]:
# Visualize performance comparison
print("📊 Performance Visualization")

# Create performance comparison chart
valid_results = {k: v for k, v in performance_results.items() if v != float('inf')}

if len(valid_results) > 1:
    plt.figure(figsize=(10, 6))
    
    methods = list(valid_results.keys())
    times = list(valid_results.values())
    
    # Bar chart
    bars = plt.bar(methods, times, color=['#ff9999', '#66b3ff', '#99ff99'][:len(methods)])
    
    # Add value labels on bars
    for bar, time_val in zip(bars, times):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                f'{time_val:.2f}s', ha='center', va='bottom')
    
    plt.title('Performance Comparison: Data Collection Methods')
    plt.ylabel('Time (seconds)')
    plt.xlabel('Collection Method')
    
    # Add speedup annotations
    if 'Sequential' in valid_results:
        baseline = valid_results['Sequential']
        for i, (method, time_val) in enumerate(valid_results.items()):
            if method != 'Sequential':
                speedup = baseline / time_val
                plt.text(i, time_val/2, f'{speedup:.1f}x\nfaster', 
                        ha='center', va='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Summary table
    print("\n📈 Performance Summary:")
    num_requests = len(PERFORMANCE_BANKS)  # Use actual number of banks tested
    perf_df = pd.DataFrame([
        {
            'Method': method,
            'Total Time (s)': f"{time_val:.2f}",
            'Avg per Request (s)': f"{time_val/num_requests:.3f}",
            'Speedup': f"{valid_results['Sequential']/time_val:.1f}x" if 'Sequential' in valid_results else 'N/A'
        }
        for method, time_val in valid_results.items()
    ])
    
    display(perf_df)
else:
    print("❌ Not enough valid results for comparison")

## 🔧 Error Handling and Validation

The library provides comprehensive error handling and data validation.

In [ ]:
# Error handling examples
print("🛡️ Error Handling Examples")

# 1. Invalid RSSD ID
print("\n1. Testing invalid RSSD ID...")
try:
    invalid_data = collect_data(
        session=basic_connection.session,
        creds=credentials,
        reporting_period="2023-12-31",
        rssd_id="invalid_id",  # Invalid: not numeric
        series="call"
    )
except ValidationError as e:
    print(f"✅ Caught ValidationError: {e}")
except ValueError as e:  # Legacy mode
    print(f"✅ Caught ValueError (legacy mode): {e}")

# 2. Empty RSSD ID
print("\n2. Testing empty RSSD ID...")
try:
    empty_data = collect_data(
        session=basic_connection.session,
        creds=credentials,
        reporting_period="2023-12-31",
        rssd_id="",  # Empty
        series="call"
    )
except (ValidationError, ValueError) as e:
    print(f"✅ Caught error for empty RSSD: {e}")

# 3. Invalid credentials
print("\n3. Testing invalid credentials...")
try:
    bad_creds = WebserviceCredentials("bad_user", "bad_pass")
    print(f"Created bad credentials (masked): {bad_creds}")
    
    # Note: This would fail on actual FFIEC service, but our mock accepts any creds
    if not DEMO_MODE:
        bad_data = collect_data(
            session=basic_connection.session,
            creds=bad_creds,
            reporting_period="2023-12-31",
            rssd_id="480228",
            series="call"
        )
    else:
        print("ℹ️ In demo mode - mock server accepts any credentials")
        
except Exception as e:
    print(f"✅ Caught authentication error: {e}")

# 4. Invalid output type
print("\n4. Testing invalid output type...")
try:
    bad_output = collect_data(
        session=basic_connection.session,
        creds=credentials,
        reporting_period="2023-12-31",
        rssd_id="480228",
        series="call",
        output_type="invalid_type"  # Invalid output type
    )
except (ValidationError, ValueError) as e:
    print(f"✅ Caught error for invalid output type: {e}")

In [ ]:
# Legacy vs New Error Mode
print("🔄 Error Mode Comparison")

from ffiec_data_connect.config import use_legacy_errors, disable_legacy_mode, enable_legacy_mode

print(f"Current legacy mode: {use_legacy_errors()}")

# Test with new error mode
print("\n📊 Testing with new error mode...")
disable_legacy_mode()
try:
    collect_data(
        session=basic_connection.session,
        creds=credentials,
        reporting_period="2023-12-31",
        rssd_id="abc123",
        series="call"
    )
except ValidationError as e:
    print(f"✅ New mode - ValidationError: {e}")
    print(f"   Error type: {type(e).__name__}")
    print(f"   Field: {e.field}")
    print(f"   Value: {e.value}")

# Test with legacy error mode
print("\n🔙 Testing with legacy error mode...")
enable_legacy_mode()
try:
    collect_data(
        session=basic_connection.session,
        creds=credentials,
        reporting_period="2023-12-31",
        rssd_id="abc123",
        series="call"
    )
except ValueError as e:
    print(f"✅ Legacy mode - ValueError: {e}")
    print(f"   Error type: {type(e).__name__}")

print(f"\nFinal legacy mode: {use_legacy_errors()}")

## 📈 Data Visualization Examples

Create visualizations from the collected financial data.

In [ ]:
# Create visualizations from the data
print("📊 Creating Data Visualizations")

# Set up the plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Use the pandas DataFrame for plotting
if len(df_pandas) > 0 and 'value' in df_pandas.columns:
    
    # 1. Bank comparison chart
    plt.figure(figsize=(15, 10))
    
    # Subplot 1: Total Assets by Bank
    plt.subplot(2, 2, 1)
    asset_data = df_pandas[df_pandas['metric'] == 'Total Assets']
    if len(asset_data) > 0:
        sns.barplot(data=asset_data, x='rssd_id', y='value', hue='reporting_period')
        plt.title('Total Assets by Bank and Period')
        plt.ylabel('Assets ($ millions)')
        plt.xlabel('Bank RSSD ID')
        plt.xticks(rotation=45)
        plt.legend(title='Reporting Period', bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Subplot 2: ROA Trend
    plt.subplot(2, 2, 2)
    roa_data = df_pandas[df_pandas['metric'] == 'ROA']
    if len(roa_data) > 0:
        for bank in roa_data['rssd_id'].unique():
            bank_roa = roa_data[roa_data['rssd_id'] == bank]
            plt.plot(bank_roa['reporting_period'], bank_roa['value'], 
                    marker='o', label=f'Bank {bank}')
        plt.title('Return on Assets (ROA) Trend')
        plt.ylabel('ROA (%)')
        plt.xlabel('Reporting Period')
        plt.xticks(rotation=45)
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    # Subplot 3: Net Income Distribution
    plt.subplot(2, 2, 3)
    income_data = df_pandas[df_pandas['metric'] == 'Net Income']
    if len(income_data) > 0:
        sns.boxplot(data=income_data, x='rssd_id', y='value')
        plt.title('Net Income Distribution by Bank')
        plt.ylabel('Net Income ($ millions)')
        plt.xlabel('Bank RSSD ID')
        plt.xticks(rotation=45)
    
    # Subplot 4: Correlation heatmap
    plt.subplot(2, 2, 4)
    # Pivot data for correlation analysis
    pivot_data = df_pandas.pivot_table(
        index=['rssd_id', 'reporting_period'], 
        columns='metric', 
        values='value', 
        aggfunc='mean'
    ).reset_index()
    
    if len(pivot_data.columns) > 3:  # At least some metrics
        numeric_cols = pivot_data.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) > 1:
            corr_matrix = pivot_data[numeric_cols].corr()
            sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0,
                       square=True, fmt='.2f')
            plt.title('Correlation Matrix of Financial Metrics')
    
    plt.tight_layout()
    plt.show()
    
    # Save the visualization
    plt.savefig('ffiec_data_visualization.png', dpi=300, bbox_inches='tight')
    print("✅ Visualization saved as 'ffiec_data_visualization.png'")
    
else:
    print("❌ No data available for visualization")

## 🧹 Cleanup and Summary

Clean up resources and summarize what we've accomplished.

In [ ]:
# Cleanup resources
print("🧹 Cleaning up resources...")

# Close connections
if 'basic_connection' in locals():
    basic_connection.close()
    print("✅ Closed basic connection")

if 'async_client' in locals():
    async_client.close()
    print("✅ Closed async client")

# Stop mock server if running
if DEMO_MODE and 'mock_server' in locals():
    mock_server.stop()
    print("✅ Stopped mock server")

# Clear SOAP cache
from ffiec_data_connect import clear_soap_cache, get_cache_stats

print(f"\nSOAP cache stats before cleanup: {get_cache_stats()}")
clear_soap_cache()
print(f"SOAP cache stats after cleanup: {get_cache_stats()}")

print("\n🎉 Cleanup completed!")

In [ ]:
# Summary of demonstration
print("📋 DEMONSTRATION SUMMARY")
print("=" * 50)

print("\n🔍 What we demonstrated:")
print("  ✅ Basic synchronous data collection")
print("  ✅ Async data collection with rate limiting")
print("  ✅ Parallel processing with sync interface")
print("  ✅ Pandas DataFrame creation and analysis")
print("  ✅ Polars DataFrame operations and performance")
print("  ✅ Data export to multiple formats (CSV, Excel, Parquet, JSON)")
print("  ✅ Performance comparison of different approaches")
print("  ✅ Comprehensive error handling and validation")
print("  ✅ Data visualization with matplotlib/seaborn")
print("  ✅ Resource management and cleanup")

print("\n💾 Files created:")
created_files = [
    'ffiec_data_pandas.csv',
    'ffiec_data_pandas.xlsx', 
    'ffiec_data_pandas.parquet',
    'ffiec_data_polars.csv',
    'ffiec_data_polars.parquet',
    'ffiec_data_polars.json',
    'ffiec_data_visualization.png'
]

total_size = 0
for file in created_files:
    if os.path.exists(file):
        size_kb = os.path.getsize(file) / 1024
        total_size += size_kb
        print(f"  📄 {file}: {size_kb:.1f} KB")

print(f"\n📊 Total files size: {total_size:.1f} KB")

if 'performance_results' in locals():
    print("\n⚡ Performance highlights:")
    for method, time_val in performance_results.items():
        if time_val != float('inf'):
            print(f"  {method}: {time_val:.2f} seconds")

print("\n🚀 Ready for production use!")
print("\n💡 Next steps:")
print("  1. Set up your FFIEC credentials as environment variables")
print("  2. Use AsyncCompatibleClient for better performance")
print("  3. Choose Polars for large datasets, Pandas for smaller ones")
print("  4. Implement proper error handling in your applications")
print("  5. Use context managers for automatic resource cleanup")

print("\n📚 Documentation: https://ffiec-data-connect.readthedocs.io/")
print("🐛 Report issues: https://github.com/call-report/ffiec-data-connect/issues")